In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import joblib, scipy

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold, RepeatedKFold, RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegressionCV, PassiveAggressiveClassifier, SGDClassifier, RidgeClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, VotingClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#data read
df = pd.read_csv('path.csv')
df.head()
df.shape
df.info()
df.describe()
df.isnull().sum()
df[df.duplicated()]
labels = df['target'].value_counts().keys()
values = df['target'].value_counts().values

In [ ]:
X = df.drop('target',axis=1)
y = df['target']

In [ ]:
scaler = StandardScaler()
features = X.columns
X = scaler.fit_transform(X)
X = pd.DataFrame(X,columns=features)
X.head()

In [ ]:
logmodel = LogisticRegressionCV(solver='liblinear')
logmodel.fit(X_train,y_train)

In [ ]:
lr_pred = logmodel.predict(X_test)
print(confusion_matrix(y_test,lr_pred))
print(classification_report(y_test,lr_pred))
plot_confusion_matrix(logmodel,X_test,y_test)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, ConfusionMatrixDisplay
from sklearn.utils import resample
import matplotlib.pyplot as plt

# Load the dataset
dataset_path = '/folder/path/.csv'
data = pd.read_csv(dataset_path)

# Drop unnecessary columns
data = data.drop(columns=['col1', 'col2', 'col3'])

# Split the dataset into features and target
target_column = 'tgt'
X = data.drop(columns=[target_column])
y = data[target_column]

# Handle categorical features (if any)
if X.select_dtypes(include=['object']).shape[1] > 0:
    X = pd.get_dummies(X, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Balance the training data
train_data = pd.concat([X_train, y_train], axis=1)
majority_class = train_data[y_train.name].value_counts().idxmax()
min_class_count = train_data[y_train.name].value_counts().min()

balanced_data = pd.concat([
    resample(train_data[train_data[y_train.name] == cls], 
             replace=False, 
             n_samples=min_class_count, 
             random_state=42)
    for cls in train_data[y_train.name].unique()
])

X_train = balanced_data.drop(columns=[y_train.name])
y_train = balanced_data[y_train.name]

# Train a Random Forest classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)

# Plot confusion matrix
ConfusionMatrixDisplay.from_estimator(model, X_test, y_test, cmap='Blues')
plt.title("Confusion Matrix")
plt.show()
